# **Course:** Introduction to Computer Vision (CS231.Q11)

**Topic:** Face Mask Classification

**Member:** 
- Nguyen Cong Phat - 23521143
- Nguyen Le Phong - 23521168
- Vu Viet Cuong - 23520213 

**Imports and Configuration**

In [1]:
import os
from pathlib import Path
import numpy as np
import joblib
from tqdm import tqdm

# Image Processing
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.feature import hog

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Optimization
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler

# --- CONFIGURATION ---
DATA_DIR = Path('/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset')
LABELS = ['WithMask', 'WithoutMask']
IMAGE_SIZE = (128, 128)

# HOG Descriptor Parameters
HOG_PARAMS = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys',
}
# Output filename
MODEL_FILENAME = 'hog8x2_rf.joblib'

**Feature Extraction & Data Loading**

In [2]:
def extract_hog_features(path: Path) -> np.ndarray:
    """Reads an image, resizes it, and returns HOG features."""
    img = imread(path)
    # Handle RGBA or Grayscale inputs robustly
    gray = rgb2gray(img) if img.ndim == 3 else img
    gray = resize(gray, IMAGE_SIZE, anti_aliasing=True)
    return hog(gray, **HOG_PARAMS)

def load_split(split: str):
    """Loads images from a specific folder (Train/Validation/Test)."""
    X, y = [], []
    for label in LABELS:
        folder = DATA_DIR / split / label
        # Loading loop with progress bar
        for img_path in tqdm(list(folder.glob('*.*')), desc=f'Loading {split}/{label}'):
            try:
                X.append(extract_hog_features(img_path))
                y.append(label)
            except Exception as e:
                print(f'-- error reading {img_path}: {e}')
    return np.vstack(X), np.array(y)

**Load Training & Validation Data**

In [3]:
print("--- Loading Training Data ---")
X_train, y_train = load_split('Train')

print("\n--- Loading Validation Data ---")
X_val, y_val = load_split('Validation')

# Encode labels (WithMask -> 0, WithoutMask -> 1)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

print(f"\nTraining Shape: {X_train.shape}")
print(f"Validation Shape: {X_val.shape}")

--- Loading Training Data ---


Loading Train/WithoutMask: 100%|██████████| 5000/5000 [01:15<00:00, 66.00it/s]



--- Loading Validation Data ---


Loading Validation/WithoutMask: 100%|██████████| 400/400 [00:05<00:00, 68.22it/s]


Training Shape: (10000, 8100)
Validation Shape: (800, 8100)


**Optuna Optimization Logic**

In [4]:
def objective(trial):
    """
    Optuna objective function to tune Random Forest hyperparameters.
    Uses warm_start to prune unpromising trials early.
    """
    # Define search space
    params = {
        'n_estimators': 10,  # Start small, increase in loop
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'random_state': 42,
        'warm_start': True,  # Essential for incremental learning loop below
        'n_jobs': 1          # Keep 1 here, parallelize via Optuna study instead
    }
    
    clf = RandomForestClassifier(**params)

    # Incremental training loop (50 -> 100 -> 150 -> 200 trees)
    # This allows us to prune bad trials before they finish full training
    for n in [50, 100, 150, 200]:
        clf.set_params(n_estimators=n)
        clf.fit(X_train, y_train_enc)
        
        # Validate
        preds = clf.predict(X_val)
        acc = accuracy_score(y_val_enc, preds)
        
        # Report to Optuna
        trial.report(acc, n)
        
        # Pruning check
        if trial.should_prune():
            raise optuna.TrialPruned()
            
    # Save the model instance to user_attrs so we can retrieve the best one later
    trial.set_user_attr("model", clf)
    return acc

**Run Optimization**

In [5]:
print("--- Starting Optuna Optimization ---")
study = optuna.create_study(
    direction='maximize',
    pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1),
    sampler=TPESampler(multivariate=True)
)

# Run optimization (n_jobs=4 uses 4 CPU cores)
study.optimize(objective, n_trials=50, n_jobs=4)

print("Best validation accuracy:", study.best_value)
print("Best parameters:", study.best_params)

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:32: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-11-29 17:43:19,531] A new study created in memory with name: no-name-570135d7-779a-40e8-bd9f-4168ad695180


--- Starting Optuna Optimization ---


[I 2025-11-29 17:43:45,615] Trial 0 finished with value: 0.9725 and parameters: {'max_depth': 26, 'min_samples_split': 14, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 0.9725.
[I 2025-11-29 17:44:06,700] Trial 4 finished with value: 0.975 and parameters: {'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 4 with value: 0.975.
[I 2025-11-29 17:44:50,970] Trial 2 finished with value: 0.97375 and parameters: {'max_depth': 5, 'min_samples_split': 20, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 4 with value: 0.975.
[I 2025-11-29 17:46:29,249] Trial 1 finished with value: 0.98125 and parameters: {'max_depth': 50, 'min_samples_split': 20, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.98125.
[I 2025-11-29 17:46:56,444] Trial 7 finished with value: 0.97375 and parameters: {'max_depth': 33, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 'log2'}. Bes

Best validation accuracy: 0.9875
Best parameters: {'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': 'sqrt'}


**Final Test & Save**

In [6]:
# Retrieve the best model from the study
best_clf = study.best_trial.user_attrs["model"]

print("\n--- Loading Test Data ---")
X_test, y_test = load_split('Test')
y_test_enc = le.transform(y_test)

# Predict
print("\n--- Evaluating ---")
y_pred = best_clf.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test_enc, y_pred))
print("\nClassification Report:")
print(classification_report(y_test_enc, y_pred, target_names=le.classes_))

# Save
joblib.dump({'model': best_clf, 'label_encoder': le}, MODEL_FILENAME)
print(f'\nModel and encoder saved to {MODEL_FILENAME}')


--- Loading Test Data ---


Loading Test/WithoutMask: 100%|██████████| 509/509 [00:07<00:00, 69.74it/s]



--- Evaluating ---
Test Accuracy: 0.9818548387096774

Classification Report:
              precision    recall  f1-score   support

    WithMask       0.97      1.00      0.98       483
 WithoutMask       1.00      0.97      0.98       509

    accuracy                           0.98       992
   macro avg       0.98      0.98      0.98       992
weighted avg       0.98      0.98      0.98       992


Model and encoder saved to hog8x2_rf.joblib
